In [1]:
import sys
sys.path.append("..")

from pathlib import Path

from pytorch_lightning.utilities.model_summary import summarize
import torch
from torch import nn

from src.intent.models import IntentClassifier
from src.intent.data_manager import IntentDataManager

In [2]:
data_manager = IntentDataManager(
    cache_dir=Path("../cache/intent"),
    max_len=128,
    batch_size=32,
    num_workers=8,
    data_dir=Path("../data/intent"),
    test_file=Path("../data/intent/test.json")
)
model = IntentClassifier(
    embeddings=data_manager.embeddings,
    hidden_size=108,
    num_layers=1,
    dropout=.5,
    bidirectional=True,
    num_class=data_manager.num_class,
    lr=1e-3,
    weight_decay=0
)

2022-03-19 07:49:28 | INFO | Vocab loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/vocab.pkl
2022-03-19 07:49:28 | INFO | Intent-2-Index loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/intent2idx.json
2022-03-19 07:49:28 | INFO | Embeddings loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/embeddings.pt
/home/jacky/.pyenv/versions/3.8.12/envs/adl-hw01/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [3]:
valid_dataloader = data_manager.get_valid_dataloader()
x, length, y = next(iter(valid_dataloader))
x.shape, length.shape, y.shape

(torch.Size([32, 24]), torch.Size([32]), torch.Size([32]))

In [4]:
embeddings = model.embedding(x)
embeddings.shape, embeddings.transpose(1,2).shape

(torch.Size([32, 24, 300]), torch.Size([32, 300, 24]))

In [5]:
conv = nn.Sequential(
    nn.Conv1d(
        in_channels=model.embedding.embedding_dim,
        out_channels=64,
        kernel_size=5,
    ),
    nn.LeakyReLU(),
    nn.MaxPool1d(kernel_size=4)
)

In [6]:
conv(embeddings.transpose(1,2)).shape

torch.Size([32, 64, 5])

In [195]:
from src.intent.models import IntentClassifier_CNN_BiLSTM

model_cnn_bilstm = IntentClassifier_CNN_BiLSTM(
    cnn_out_channels=64,
    cnn_kernel_size=1,
    pool_size=4,
    embeddings=data_manager.embeddings,
    hidden_size=155,
    num_layers=1,
    dropout=.5,
    bidirectional=True,
    num_class=data_manager.num_class,
    lr=1e-3,
    weight_decay=0
)

In [196]:
summarize(model_cnn_bilstm)

  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 1.6 M 
1 | rnn       | LSTM             | 274 K 
2 | fc        | Sequential       | 143 K 
3 | loss      | CrossEntropyLoss | 0     
4 | cnn       | Sequential       | 19.3 K
-----------------------------------------------
436 K     Trainable params
1.6 M     Non-trainable params
2.1 M     Total params
8.327     Total estimated model params size (MB)

In [38]:
model_cnn_bilstm(x, length).shape

torch.Size([32, 150])